In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# Image Generation from Audio
from pathlib import Path
from IPython.display import Audio
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
from utils import read_file, transform_path

import time

In [ ]:
DATA = Path('D:/Experiments/data')

# these folder must be in place
NSYNTH_AUDIO = DATA/'nsynth_audio'
TRAIN_AUDIO_PATH = NSYNTH_AUDIO/'train/audio'
VALID_AUDIO_PATH = NSYNTH_AUDIO/'valid/audio'

# these folders will be created
NSYNTH_IMAGES = DATA/'nsynth_images'
TRAIN_IMAGE_PATH = NSYNTH_IMAGES/'train'
VALID_IMAGE_PATH = NSYNTH_IMAGES/'valid'

In [ ]:
train_acoustic_fnames = [f.name for f in TRAIN_AUDIO_PATH.iterdir()
                         if 'acoustic' in f.name]
valid_acoustic_fnames = [f.name for f in VALID_AUDIO_PATH.iterdir()
                         if 'acoustic' in f.name]
len(train_acoustic_fnames), len(valid_acoustic_fnames)

In [ ]:
fn = train_acoustic_fnames[8]; fn

In [ ]:
Audio(str(TRAIN_AUDIO_PATH/fn))

In [ ]:
x, sr = read_file(fn, TRAIN_AUDIO_PATH)
x.shape, sr, x.dtype

In [ ]:
def log_mel_spec_tfm(fname, src_path, dst_path):
    x, sample_rate = read_file(fname, src_path)
    
    n_fft = 1024
    hop_length = 256
    n_mels = 40
    fmin = 20
    fmax = sample_rate / 2 
    
    mel_spec_power = librosa.feature.melspectrogram(x, sr=sample_rate, n_fft=n_fft, 
                                                    hop_length=hop_length, 
                                                    n_mels=n_mels, power=2.0, 
                                                    fmin=fmin, fmax=fmax)
    mel_spec_db = librosa.power_to_db(mel_spec_power, ref=np.max)
    dst_fname = dst_path / (fname[:-4] + '.png')
    plt.imsave(dst_fname, mel_spec_db)

In [ ]:
log_mel_spec_tfm(fn, TRAIN_AUDIO_PATH, Path('.'))
img = plt.imread(fn[:-4] + '.png')
plt.imshow(img, origin='lower');

In [ ]:
start = time.time()
print("hello")
end = time.time()
print(end - start)

In [ ]:
# Train files
start = time.time()
print("Training image generating start")
transform_path(TRAIN_AUDIO_PATH, 
               TRAIN_IMAGE_PATH, 
               log_mel_spec_tfm, 
               fnames=train_acoustic_fnames, 
               delete=True)
end = time.time()
print('finished ! and this took', end - start)

In [ ]:
# Valid files
start = time.time()
print("Validation image generating start")
transform_path(VALID_AUDIO_PATH, 
               VALID_IMAGE_PATH, 
               log_mel_spec_tfm, 
               fnames=valid_acoustic_fnames, 
               delete=True)
end = time.time()
print('finished ! and this took', end - start)